# Qiskit Multirunner

### Imports

In [116]:
import json
import csv

import qiskit
from qiskit import QuantumCircuit, QuantumRegister
import qiskit_aer
from mqt import ddsim
from qiskit_ibm_runtime import SamplerV2 as Sampler, QiskitRuntimeService

from qiskit_aer import AerSimulator
from qiskit_qulacs import QulacsProvider

### Sample Circuit

In [117]:
def get_sample_circuit() -> QuantumCircuit:
    qubits = QuantumRegister(1)
    qc = QuantumCircuit(qubits)
    qc.h(qubits[0])
    qc.measure_all()
    return qc

In [118]:
# Display our sample circuit
qc = get_sample_circuit()
qc.draw()

┌───┐ ░ ┌─┐
   q24: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0

### Grover circuit

In [119]:
from qiskit_algorithms import Grover
from qiskit.primitives import Sampler
from qiskit_algorithms import AmplificationProblem

def get_amp_circuit():
    # the state we desire to find is '11'
    good_state = ["11"]

    # specify the oracle that marks the state '11' as a good solution
    oracle = QuantumCircuit(2)
    oracle.cz(0, 1)

    # define Grover's algorithm
    problem = AmplificationProblem(oracle, is_good_state=good_state)
    return problem.grover_operator.decompose()

In [120]:
qc = get_amp_circuit()
qc.draw()

global phase: π
            ┌───┐┌───┐          ┌───┐┌───┐     
state_0: ─■─┤ H ├┤ X ├───────■──┤ X ├┤ H ├─────
          │ ├───┤├───┤┌───┐┌─┴─┐├───┤├───┤┌───┐
state_1: ─■─┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├┤ X ├┤ H ├
            └───┘└───┘└───┘└───┘└───┘└───┘└───┘

### Variational Quantum Eigensolver

In [121]:
import numpy as np
import networkx as nx

num_nodes = 4
w = np.array(
    [[0.0, 1.0, 1.0, 0.0], [1.0, 0.0, 1.0, 1.0], [1.0, 1.0, 0.0, 1.0], [0.0, 1.0, 1.0, 0.0]]
)
G = nx.from_numpy_array(w)

from qiskit.quantum_info import Pauli, SparsePauliOp


def get_operator(weight_matrix):
    r"""Generate Hamiltonian for the graph partitioning
    Notes:
        Goals:
            1 Separate the vertices into two set of the same size.
            2 Make sure the number of edges between the two set is minimized.
        Hamiltonian:
            H = H_A + H_B
            H_A = sum\_{(i,j)\in E}{(1-ZiZj)/2}
            H_B = (sum_{i}{Zi})^2 = sum_{i}{Zi^2}+sum_{i!=j}{ZiZj}
            H_A is for achieving goal 2 and H_B is for achieving goal 1.
    Args:
        weight_matrix: Adjacency matrix.
    Returns:
        Operator for the Hamiltonian
        A constant shift for the obj function.
    """
    num_nodes = len(weight_matrix)
    pauli_list = []
    coeffs = []
    shift = 0

    for i in range(num_nodes):
        for j in range(i):
            if weight_matrix[i, j] != 0:
                x_p = np.zeros(num_nodes, dtype=bool)
                z_p = np.zeros(num_nodes, dtype=bool)
                z_p[i] = True
                z_p[j] = True
                pauli_list.append(Pauli((z_p, x_p)))
                coeffs.append(-0.5)
                shift += 0.5

    for i in range(num_nodes):
        for j in range(num_nodes):
            if i != j:
                x_p = np.zeros(num_nodes, dtype=bool)
                z_p = np.zeros(num_nodes, dtype=bool)
                z_p[i] = True
                z_p[j] = True
                pauli_list.append(Pauli((z_p, x_p)))
                coeffs.append(1.0)
            else:
                shift += 1

    return SparsePauliOp(pauli_list, coeffs=coeffs), shift


qubit_op, offset = get_operator(w)

In [122]:
from qiskit.primitives import Sampler
from qiskit.quantum_info import Pauli
from qiskit.result import QuasiDistribution

from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA

from qiskit_algorithms.utils import algorithm_globals

sampler = Sampler()


def sample_most_likely(state_vector):
    """Compute the most likely binary string from state vector.
    Args:
        state_vector: State vector or quasi-distribution.

    Returns:
        Binary string as an array of ints.
    """
    if isinstance(state_vector, QuasiDistribution):
        values = list(state_vector.values())
    else:
        values = state_vector
    n = int(np.log2(len(values)))
    k = np.argmax(np.abs(values))
    x = bitfield(k, n)
    x.reverse()
    return np.asarray(x)


algorithm_globals.random_seed = 10598

optimizer = COBYLA()
qaoa = QAOA(sampler, optimizer, reps=2)

result = qaoa.compute_minimum_eigenvalue(qubit_op)

/var/folders/0q/dj3wgc5576l38_bq_5tdyvf40000gn/T/ipykernel_1179/334686316.py:10: DeprecationWarning: The class ``qiskit.primitives.sampler.Sampler`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseSamplerV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Sampler` class is `StatevectorSampler`.
  sampler = Sampler()


  ### VQE (Variational Quantum Eigensolver)

In [124]:
from qiskit_algorithms import VQE
from qiskit_algorithms.utils import algorithm_globals
from qiskit_aer.primitives import Estimator as AerEstimator
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.optimizers import SPSA

counts = []
values = []

def store_intermediate_result(eval_count, parameters, mean, std):
    counts.append(eval_count)
    values.append(mean)

iterations = 125
ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
spsa = SPSA(maxiter=iterations)
seed = 170
algorithm_globals.random_seed = seed

noiseless_estimator = AerEstimator(
    run_options={"seed": seed, "shots": 1024},
    transpile_options={"seed_transpiler": seed},
)


H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

print(f"Number of qubits: {H2_op.num_qubits}")

vqe = VQE(noiseless_estimator, ansatz, optimizer=spsa, callback=store_intermediate_result)
result = vqe.compute_minimum_eigenvalue(operator=H2_op)

print(f"VQE on Aer qasm simulator (no noise): {result.eigenvalue.real:.5f}")
print(f"Delta from reference energy value is {(result.eigenvalue.real - ref_value):.5f}")

Number of qubits: 2
VQE on Aer qasm simulator (no noise): -1.85160
Delta from reference energy value is 0.00567


### Qiskit AER Execution

In [ ]:
def run_on_aer(qc: QuantumCircuit):
    backend = AerSimulator()
    result = backend.run(qc, shots=1024).result()
    return result
job = run_on_aer(get_amp_circuit())
job

Result(backend_name='aer_simulator', backend_version='0.16.3', qobj_id='', job_id='a01f1e7b-cdd4-4988-b5ee-7031a706fb85', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(), header=QobjExperimentHeader(creg_sizes=[], global_phase=3.141592653589793, memory_slots=0, n_qubits=2, name='Q', qreg_sizes=[['state', 2]], metadata={}), status=DONE, seed_simulator=2971035555, metadata={'num_bind_params': 1, 'runtime_parameter_bind': False, 'parallel_state_update': 4, 'parallel_shots': 1, 'batched_shots_optimization': False, 'remapped_qubits': False, 'active_input_qubits': [], 'device': 'CPU', 'time_taken': 2.536e-05, 'measure_sampling': False, 'num_clbits': 0, 'max_memory_mb': 8192, 'input_qubit_map': [], 'num_qubits': 0, 'method': 'stabilizer', 'required_memory_mb': 0}, time_taken=2.536e-05)], date=2025-04-10T13:28:28.375837, status=COMPLETED, header=None, metadata={'time_taken_parameter_binding': 4.497e-06, 'max_memory_mb': 8192, 'time_ta

In [ ]:
job.get_counts()

QiskitError: 'No counts for experiment "0"'

### Qiskit MQT Execution

In [ ]:
def run_on_mqt(qc: QuantumCircuit):
    backend = ddsim.DDSIMProvider().get_backend("qasm_simulator")
    result = backend.run(qc, shots=1024).result()
    return result

job = run_on_mqt(get_sample_circuit())

In [ ]:
job.get_counts()

{'0': 531, '1': 493}

### Qiskit Qulacs Execution

In [ ]:
def run_on_qulacs(qc: QuantumCircuit):
    backend = QulacsProvider().get_backend('qulacs_simulator')
    result = backend.run(qc).result()
    return result

run_on_qulacs(get_sample_circuit())

/var/folders/0q/dj3wgc5576l38_bq_5tdyvf40000gn/T/ipykernel_1179/180786299.py:2: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = QulacsProvider().get_backend('qulacs_simulator')
/Users/mews/Documents/repos/qiskit-multirunner/.conda/lib/python3.11/site-packages/qiskit_qulacs/adapter.py:246: UserWarning: qiskit_qulacs.adapter: The Barrier instruction is not supported by Qiskit-Qulacs and has not been added to the circuit.
  warnings.warn(
/Users/mews/Documents/repos/qiskit-multirunner/.conda/lib/python3.11/site-packages/qiskit_qulacs/adapter.py:246: UserWarning: qiskit_qulacs.adapter: The Measure instruction is not supported by Qiskit-Qulacs and has not bee

Result(backend_name='qulacs_simulator', backend_version='0.1.0', qobj_id='0', job_id='78862cea-b5e9-4ac2-bba7-2c84bf760303', success=True, results=[ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(statevector=[0.70710678+0.j 0.70710678+0.j]), status=JobStatus.DONE)], date=None, status=JobStatus.DONE, header=None, time_taken=5.0067901611328125e-05)

In [ ]:
def log_1000_jobs_per_sumulator():
    samples = []
    circuit = get_amp_circuit()
    for i in range(0,999):
        execution_dict = run_on_aer(circuit).to_dict()
        samples.append(execution_dict)
    for i in range(0,999):
        execution_dict = run_on_mqt(circuit).to_dict()
        samples.append(execution_dict)
    for i in range(0,999):
        execution_dict = run_on_qulacs(circuit).to_dict()
        samples.append(execution_dict)
    return samples

def jobs_to_csv(samples: list, target_file: str):
    file = open(target_file, 'r+')
    #get a sample of the dataset for write parameters
    sample = samples[0]
    #write the type header
    header = ','.join(sample.keys())
    file.write(header)

    writer = csv.DictWriter(file, fieldnames=list(sample.keys()))

    for i in samples:
        writer.writerow(i)

    file.close()

    pass

execs = log_1000_jobs_per_sumulator()
jobs_to_csv(execs, '../output/simulations.csv')

/var/folders/0q/dj3wgc5576l38_bq_5tdyvf40000gn/T/ipykernel_1179/180786299.py:2: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = QulacsProvider().get_backend('qulacs_simulator')


# Real-hardware execution

Configure the following with your account in order to execute jobs in actual IBM hardware, it implies a compute time cost, so you should probably be cautious about running this multiple times.

In [ ]:
#please put your account info here
QiskitRuntimeService.save_account(
  token='',
  channel="ibm_quantum", # `channel` distinguishes between different account types
  overwrite=True # To change the account again for every execution, useful if you're using multiple tokens
)

def run_on_hw(qc: QuantumCircuit):
    service = QiskitRuntimeService()
    # Get a backend
    backend = service.least_busy(operational=True, simulator=False)
    
    # Define Sampler
    sampler = Sampler(mode=backend)
    
    # Run calculation
    job = sampler.run([qc])
    
    result = job.result()
    return result

run_on_hw(get_sample_circuit())

InvalidAccountError: "Invalid `token` value. Expected a non-empty string, got ''."

In [ ]:
import qiskit.compiler


def run_on_hw_transpiled(qc: QuantumCircuit):
    
    service = QiskitRuntimeService()
    # Get a backend
    backend = service.least_busy(operational=True, simulator=False)
    qc = qiskit.compiler.transpile(qc, backend=backend)
    # Define Sampler
    sampler = Sampler(mode=backend)
    
    # Run calculation
    job = sampler.run([qc])
    
    result = job.result()
    return result

run_on_hw(get_sample_circuit())

IBMInputValueError: 'The instruction h on qubits (0,) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/guides/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/guides/primitives-examples) to see this coupled with operator transformations.'

: 

: 

: 

: 